In [1]:
import coffea
from git import Repo
import awkward
import numpy as np
import numba
import uproot
import dask_awkward as dak
from dask_awkward import dask_method
import hist
import mplhep as hep
import matplotlib.pyplot as plt
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")


_______________________________________
	Current Configuration
---------------------------------------
Coffea Version:  0.1.dev3583+ge06c4b8
Branch: 	 master
Modified by: 	 Prayag Yadav
_______________________________________


In [16]:
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
#test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
test_file = './test_Spring2021.root'
file = uproot.open(test_file)

events = NanoEventsFactory.from_root(
    test_file+":events",
    # entry_stop=100000,
    schemaclass= FCC.get_schema(version="latest"),
    delayed = False,
    metadata=file["metadata"].arrays()
).events()

file.close()

In [17]:
events.Particle.get_parents

<MCTruthParticleArray [[[], [], [{...}], ..., [], []], ...] type='100 * var...'>

# Get all the daughters

- The implementation I have come up looks a bit shabby. I wonder if there is a better an more efficient way.
- `get_daughters_index` generates a doubly-nested array of all the index ranges from begin to end
- For example, if the first `Particle` of the first has daughters that `begin` with `0` and `end` with `1`, then, the range of indices is `range(0,1)` = `[0]`. the `Particeidx1` value corresponding to this is `[2]`.

In [18]:
events.Particle.get_daughters_index

<Array [[[2], [3], [4, ...], ..., [], []], ...] type='100 * var * var * int...'>

- Finally, to get the actual daughter MC particle, one can simply call `get_daughters`
- For Example, if the the first element of the first event in the output of `get_daughters_index` was `[2]`, that would produce the array `[<Particle at the index 2 in the Particle collection>]`

In [19]:
events.Particle.get_daughters

<MCTruthParticleArray [[[{PDG: 11, ...}], ..., []], ...] type='100 * var * ...'>

# Get all the Parents

- Very similar to daughters, parents can be generated using very similar functions

In [20]:
events.Particle.get_parents_index

<Array [[[], [], [0], ..., [276], [], []], ...] type='100 * var * var * int...'>

In [21]:
events.Particle.get_parents

<MCTruthParticleArray [[[], [], [{...}], ..., [], []], ...] type='100 * var...'>

# Definition of the functions

```python
def map_index_to_array(array, index, axis=1):
    '''
    DESCRIPTION: Creates a slice of input array according to the input index.
    INPUTS: array (Singly nested)
            index (Singly or Doubly nested)
            axis (By default 1, use axis = 2 if index is doubly nested )
    EXAMPLE:
            a = awkward.Array([
                [44,33,23,22],
                [932,24,456,78],
                [22,345,78,90,98,24]
            ])

            a_index = awkward.Array([
                [0,1,2],
                [0,1],
                []
            ])

            a2_index = awkward.Array([
                [[0],[0,1],[2]],
                [[0,1]],
                []
            ])
            >> map_index_to_array(a, a_index)
                [[44, 33, 23],
                 [932, 24],
                 []]
                ---------------------
                type: 3 * var * int64
            >> map_index_to_array(a, a2_index, axis=2)
                [[[44], [44, 33], [23]],
                 [[932, 24]],
                 []]
                ---------------------------
                type: 3 * var * var * int64

    '''
    if axis==1:
        return array[index]
    elif axis==2:
        axis2_counts_array = awkward.num(index, axis=axis)
        flat_axis2_counts_array = awkward.flatten(axis2_counts_array, axis=1)
        flat_index = awkward.flatten(index, axis=axis)
        trimmed_flat_array = array[flat_index]
        trimmed_array = awkward.unflatten(trimmed_flat_array, flat_axis2_counts_array, axis=1)
        return trimmed_array
    else:
        raise AttributeError('Only axis = 1 or axis = 2 supported at the moment.')

@awkward.mixin_class(behavior)
class MCTruthParticle(MomentumCandidate, base.NanoCollection):
    """Generated Monte Carlo particles."""


    @numba.njit
    def index_range_numba_wrap(self, begin_end, builder):
        for ev in begin_end:
            builder.begin_list()
            for j in ev:
                builder.begin_list()
                for k in range(j[0],j[1]):
                    builder.integer(k)
                builder.end_list()
            builder.end_list()
        return builder

    def index_range(self, begin, end):
        begin_end = awkward.concatenate((begin[:,:,numpy.newaxis],end[:,:,numpy.newaxis]),axis=2)
        if awkward.backend(begin) == "typetracer" or awkward.backend(end) == "typetracer":
            # here we fake the output of numba wrapper function since
            # operating on length-zero data returns the wrong layout!
            awkward.typetracer.length_zero_if_typetracer(begin_end) # force touching of the necessary data
            return awkward.Array(awkward.Array([]).layout.to_typetracer(forget_length=True))
        return self.index_range_numba_wrap(begin_end, awkward.ArrayBuilder()).snapshot()

    #Daughters
    @dask_property
    def get_daughters_index(self):
        ranges = self.index_range(self.daughters.begin, self.daughters.end)
        return awkward.values_astype(map_index_to_array(self._events().Particleidx1.index, ranges, axis=2), "int64")

    @get_daughters_index.dask
    def get_daughters_index(self, dask_array):
        ranges = dask_awkward.map_partitions(self.index_range, dask_array.daughters.begin, dask_array.daughters.end)
        return awkward.values_astype(map_index_to_array(dask_array._events().Particleidx1.index, ranges, axis=2), "int64")

    @dask_property
    def get_daughters(self):
        return map_index_to_array(self, self.get_daughters_index, axis=2)

    @get_daughters.dask
    def get_daughters(self, dask_array):
        return map_index_to_array(dask_array, dask_array.get_daughters_index, axis=2)

    #Parents
    @dask_property
    def get_parents_index(self):
        ranges = self.index_range(self.parents.begin, self.parents.end)
        return awkward.values_astype(map_index_to_array(self._events().Particleidx0.index, ranges, axis=2), "int64")

    @get_parents_index.dask
    def get_parents_index(self, dask_array):
        ranges = dask_awkward.map_partitions(self.index_range, dask_array.parents.begin, dask_array.parents.end)
        return awkward.values_astype(map_index_to_array(dask_array._events().Particleidx0.index, ranges, axis=2), "int64")

    @dask_property
    def get_parents(self):
        return map_index_to_array(self, self.get_parents_index, axis=2)

    @get_parents.dask
    def get_parents(self, dask_array):
        return map_index_to_array(dask_array, dask_array.get_parents_index, axis=2)
```

#### Function : index_range

- `index_range` creates the ranges of indices for each begin and end index 

In [22]:
events.Particle.daughters.begin

<Array [[0, 1, 2, 4, ..., 576, 576, 576], ...] type='100 * var * uint32[par...'>

In [23]:
events.Particle.daughters.end

<Array [[1, 2, 4, 6, ..., 576, 576, 576], ...] type='100 * var * uint32[par...'>

In [24]:
ranged = events.Particle.index_range(events.Particle.daughters.begin, events.Particle.daughters.end )
ranged

<Array [[[0], [1], [2, ...], ..., [], []], ...] type='100 * var * var * int64'>

#### Once the ranges are created, one can map them to the desired array to get the required slice of the same array

#### `ranged` array has ranges for the elements in `events.Particleidx1`, so we have to map it to events.Particleidx1 to get the `daughter_index`

In [25]:
from coffea.nanoevents.methods.fcc import map_index_to_array

In [27]:
daughter_index = map_index_to_array(events.Particleidx1.index, ranged, axis=2)
daughter_index

<Array [[[2], [3], [4, ...], ..., [], []], ...] type='100 * var * var * int...'>

#### This `daughter_index` can be in turn mapped to the `events.Particle` to get the actual daughter MC particles

In [28]:
daughters = map_index_to_array(events.Particle, daughter_index, axis=2)
daughters

<MCTruthParticleArray [[[{PDG: 11, ...}], ..., []], ...] type='100 * var * ...'>

In [29]:
daughters.PDG

<Array [[[11], [-11], [...], ..., [], []], ...] type='100 * var * var * int...'>

#### One can perform similar operations to get parents too (use `events.Particleidx0` instead of `events.Particleidx1`)

# Further Comments

- This works really well and fast for `delayed=False` mode
- Currently, this doesn't work for `delayed=True`, but I think that should be easily done with some efforts